# Module 3: Locating KITT Using Audio Communication

__Learning objectives__ The following is learned and practiced in this module:
- The importance of good auto-correlation properties of your bit-code
- Channel estimation and TDOA from recorded signals
- item Location estimation algorithms based on TDOA

    
__Preparation__
- Read this module
- Review your TDOA and deconvolution algorithms from the EE2T11 Telecommunications A(??@B update) practical
- Study Appendix C (??@B update)
- Make sure your Python IDE is ready to go!


__Time duration__ Four lab sessions and four preparation/reporting sessions at home
    
__What is needed:__ The following
- KITT
- Laptop/PC running Python
- Your TDOA and deconvolution algorithms from EE2T11 Telecommunications A
- On Brightspace: 7+3 pre-recorded audio recordings.





KITT must be located in its field and then directions must be determined to navigate to the final destination. In the previous modules your colleagues are developing scripts to communicate with KITT. They will add functionality to read the audio signals from the microphones located around the field, and you should use these to locate the car. It is recommended for all group members to read Modules 1 and 2 to have a better understanding of how everything should work together.

For the localization, we will use (real-time) recordings of the beacon signal at the various microphones, deconvolve these using a reference signal recording, and determine the relative time delays from the resulting channel estimates. (It is assumed that you have working channel estimation algorithms from the
EE2T11 Telecommunications A (??@B update) course lab.) Depending on the distance to each microphone, the signal transmitted by KITT’s beacon arrives a little bit earlier or later, and you can convert that into physical distances. For each pair of microphones, we can compute this time difference of arrival (TDOA), or the physical difference in propagation distance. If you have measurements from enough microphones, then you can calculate the location of KITT in the field.

To get you started, 7 recodings with known locations and a reference recording taken close to the microphone will be provided in this task. This can be used to develop and test your algorithms.

At the end of this Module, you will have developed a script to locate KITT within the field with reason- able accuracy, and you will do so using the data recorded by the microphones located along the field. You will also have tested and verified the accuracy and robustness of your solution.


## Pre-recorded data

These recordings have locations randomly distributed across the field. An example recording with the location are, 

|   x   |   y   |
|-------|-------|
|  64   |   40  |
|  82   |  399  |
|  109  |   76  |
|  143  |  296  |
|  150  |  185  |
|  178  |  439  |
|  232  |  275  |

*Table 1: Locations of the given recordings (cm)*

The x and y axe are defined as follows, where the numbers refer to the microphone index:

```{figure} axisdef.png
---
height/width: 150px
name: mic-figure
---
Microphone Axis definition
```
You can assume these positions for the microphones. Please note the different height of microphone 5.

|Microphone|   x   |  y  |  z  |
|-------|-------|-------|-------|
|  1    |  0    | 0     |   50  |
|  2    |  0    | 480   |   50  |
|  3    |  480  | 480   |   50  |
|  4    |  480  | 0     |   50  |
|  5    |  0    | 240   |   80  |

*Table 2: Location of the microphones (cm)*


## Background Knowledge

We will study channel estimation: how do we measure the impulse response from a microphone to a loudspeaker? The specific topics you'll learn in this labday are channel estimation in time-domain (matched filter) and frequency domain. In Labday 4, we will test all this on audio signals.

In [ ]:
import matplotlib.pyplot as plt           # For plotting purposes
import numpy as np                        # For convolution function
import scipy
from scipy import signal                  # For filter function
from scipy.fft import fft, ifft           # For fft and ifft
import math                               # For numerical approximation of pi

## Introduction ##

In the EE2T11 Telecommunication A course lab (??@B update), you developed algorithms for channel estimation for 2 microphone signals, both receiving a beacon (training) signal. In the present module, we extend this to 5 microphones and use this to locate the car.

The channel estimation problem is the following: Suppose we transmit a known signal $x[n]$ over a communication channel, and measure the result $y[n]$.  The channel acts as a
filter, which we will assume to be linear and time-invariant.
Therefore, the measured signal is a convolution of the transmitted
signal by the channel impulse response $h[n]$, such that
$y[n] = h[n] \ast x[n]$.  Knowing the transmitted signal $x[n]$, can
we recover the impulse response of the communication channel from
$y[n]$?  This is essentially an inversion problem.


To estimate $h[n]$, three alternative algorithms were described:
- __ch1__ Deconvolution in the time domain: Involves matrix inversion. It is computationally complex and requires lots of memory (easily more than what the available PCs can handle).

- __ch2__ The matched filter: Avoids the matrix inversion. It is equal to computing the cross-correlation of
$y[n]$ with $x[n]$. As cross-correlation is equivalent to a convolution with a reverse signal $x[−n]$, it can be computed efficiently using the FFT. The resulting channel estimate is equal to the true
channel convolved with the autocorrelation of the pulse, $x[n] ∗ x[−n]$. Therefore, its performance
heavily depends on having the correct (i.e., accurate) “reference” or “training” signal to correlate
your measurements with, and the signal should have good autocorrelation properties: $x[n] ∗ x[−n]$
should be close to a delta spike. Large sidelobes will lead to confusion.
- __ch3__ Deconvolution in the frequency domain: Involves the FFT. It computes the same channel as via
deconvolution in the time domain but is much more efficient: convolution in time becomes point-
wise multiplication in frequency, hence for deconvolution in the frequency domain, we only need
a pointwise division, followed by an IFFT to obtain the time-domain channel impulse response.

```{admonition} Note

FFT and IFFT lead to periodicities (the result is cyclic, and samples $x[n]$ at negative time reappear
at the “large n” part of the signal). Since this method is similar to ch1 (but much more efficient),
you can view this method as a matched filter, followed by a correction step that “inverts” the effect
of the autocorrelation of the transmit pulse. However, since we should not divide by zero, you will
also need to implement a “threshold” to set non-invertible or very small frequency coefficients of
$x[n]$ to zero, which otherwise will lead to noise amplification. The performance depends on this
threshold, which has to be chosen heuristically.

```
It is assumed that you have working algorithms for estimating channel responses. You developed these during the EE2T11 Telecommunication A practicum (??@B). We recommend using ch3: deconvolution in the
frequency domain.

In each case, a reference signal $x[n]$ is required. We recommend using a recording close to the beacon because then $x[n]$ includes the loudspeaker and microphone responses as well. For the pre-made recordings, a high-quality reference is available.


The deconvolution algorithm gives a channel estimate for the beacon path to each microphone. After this, we detect the first incoming path, which, assuming Line of Sight (LOS), corresponds to the propagation delay of the car beacon to each microphone. Unfortunately, we do not know the transmit time, so we only obtain relative propagation delays. If we take the difference of microphone delays, this unknown transmit time is eliminated, so we can obtain time difference of arrival (TDOA) samples for each microphone pair.
In the next sections, we will use these to locate the car.



## Deconvolution ##

The first step is to get your deconvolution algorithm operational and tested. Do these tasks with the given
recordings from Brightspace.

**Task 1**  Using the reference signal provided and the algorithms you developed at the EE2T11 Telecom-
munications A practical, deconvolve the recordings to get the channel impulse response for each
microphone. You will need to segment the received data into individual pulses; for the moment,
you can do that manually. For a few measured locations, plot examples of the segmented data and the deconvolved channels
(e.g., 10 plots per recording: the inputs and outputs of your deconvolution algorithm).

**Task 2**  From the peaks in the channel estimates, determine the time of arrivals (TOAs) and store these
in a table. Check the accuracies of these estimates. Hint: Obviously, you want to compare these
estimates to your true TOAs. The problem is that the transmission time is unknown, so a direct
comparison is impossible. You can either compare TDOAs (the pairwise differences of TOAs) or
introduce a single unknown parameter (the time of transmission) and develop an error measure
that is insensitive to it. This is harder, but you might find literature on this.

**Task 3**  Once you are satisfied with the performance of the basic algorithm, extend your code with ways
to automatically segment the received data and find the beginning of a pulse. Hint: A useful
function is findpeaks, which allows you to implement criteria for finding the first strong peak in
a pulse sequence.

**Deliverable** In the midterm report, document your algorithms, show examples of measurements/channel
estimates, and include a subsection on testing, showing your findings and accuracies.

As plots, we suggest to show an entire recording, and then one where you zoom in on the short
segment that you give to ch3. Of the resulting channel estimate, show the entire result, and then
zoom in on the interesting part where are the peaks. Remember that for negative delays, the peak
of interest will be at the “large n” part of your estimate.

## LOCALIZATION USING TDOA INFORMATION ##

Now we arrive at the main question studied in this module: how can we locate the car using the TDOA
estimates? With 5 microphones, we can compute the TDOAs between all pairs of microphones and
obtain 10 TDOA pairs.
Study Appendix C: it shows a basic algorithm to compute the (x, y) location of the car based on the
measured TDOAs. This algorithm can be extended to fit our situation: In our application, we have the
audio beacon on the car at a certain height above the ground (we define this as z = 0), but the microphones
are placed on stands at a different height. You can assume that the height difference between the audio
beacon of the car and the microphones is known.

## Localization Class

Now, it is time to start setting up a processing pipeline. The localization class you will develop will
take a 5-channel recording of input and return the x and y coordinates of the car. Below is an outline
of a localization class you could use. Note that this is to help you get started, and you will have to add
methods and logic yourself. Look at the comments inside of the class for an explanation.
(@B and @Mano add this to a separate ;pyhton module)




In [ ]:
class Localization:
    def __init__(self, recording, debug=False):
    # Store the recordings
    # Load the reference signal from memory
    # x_car, y_car = self.localization()
    def localization(self):
    # Split each recording into individual pulses
    # Calculate TDOA between different microphone pairs
    # Run the coordinate_2d using the calculated TDOAs
    def TDOA(self, rec1, rec2):
    # Calculate channel estimation of each recording using ch2 or ch3
    # Calculate TDOA between two recordings based on peaks
    # in the channel estimate
    @staticmethod
    def ch3(x, y):
    # Channel estimation
    def coordinate_2d(self, D12, D13, D14):
    # Calculate 2D coordinates based on TDOA measurements
    # using the linear algebra given before
if __name__ == "__main__":
# Main block for testing
# Read the .wav file
# Localize the sound source
# Present the results

(?? @B ans @Mano complete the code but break it into smaller section and provide the sample recordings and some test functions to check it, this needs to be very comprehensive.
)
(?? @B change the midterm, break them into task and provide a suggestion schedule)

## Localization algorithm assignments

**Task 1** Add your favorite channel estimation function from the Telecommunications Lab. Test this by
calculating some channel estimates of the given recordings.

**Task 2** Develop a test code using Pythagoras’ theorem that takes an (x, y) position as input and calculates
the TDOA that you would observe from this position. This involves calculating the distance from
the given point to each microphone. This will help you debug both your TDOA function and your
`coordinate_2d` function.

**Task 3** Complete the TDOA pipeline. It should return the time difference between all pairs of micro-
phones. Compare with results calculated by the test code from task 2.
(?? @B and @Mano maybe show an example of the standard output)

**Task 4** Implement the `coordinate_2d` method and test the module individually before connecting it
to other modules. Hint: If you know the true location, you can calculate the error. But since
the TDOA vector is redundant, your function from Task 2 can also be used to generate an error
estimate, based on your estimated position, even if you do not know your true position.

**Deliverable** Document the accuracy of your localization algorithm. Is this acceptable for your appli-
cation? As an illustration, you can also plot the room and show the true locations and estimated
locations. For the midterm report, document your algorithm and the results of the tests on simu-
lated and given test data. Also report on the given test data with unknown position: what positions
do you estimate? Finish with a conclusion that summarizes the accuracy that can be expected and
the reliability (i.e., how often it succeeds in finding a reliable location).

## Mid-term assessment 3 and report

In week 4, you will have to showcase the functionality of your localization script to your assigned TA.
You should demonstrate proper localization of the car on the 3 recordings with unknown coordinates.
After you pass this assessment, you are ready to document your results in your midterm report. A
detailed report is required, covering the approach, implementation, testing and results, as mentioned
above. Please review Chapter 7 for guidelines.

## Optional Extension

If you finish the basic assignment quickly and want to challenge yourself further, try adding additional
functionality to the program. For example, you could look at one of the following aspects:

- The current set of linear equations in Appendix C does not consider the height difference between
the microphones and the car. This leads to a slight offset. Adding in the 5th microphone at a
different height can fix this. Use logic and maths to adjust the matrices to accommodate this extra
microphone and the z-axis.
- The provided method in Appendix C is simple but unreliable for certain locations. What happens
if the distance of the car to two microphones is equal (symmetry positions)? In that case, one
column of the matrix that you try to invert is zero. During System Integration, you can search the
literature and try to implement more advanced algorithms, e.g.,
    - Stephen Bancroft, “An algebraic solution of the GSP equations”, IEEE Transactions on
Aerospace and Electronic Systems, vol.21, no.7, pp.56-59, January 1985.
    - Amir Beck, Petre Stoica, and Jian Li, “Exact and Approximate Solutions of Source Localiza-
    tion Problems”, IEEE Transactions on Signal Processing, vol.56, no.5, pp. 1770-1778, May
    2008.
    The literature on this topic is actually very rich. The latter paper gives a good overview. You can
also try to solve for the true solution using the nonlinear Least Squares optimization toolboxes.
In previous years, some students have implemented a grid search, in which the room is partitioned
into a dense grid of possible positions, and each location is tested against the TDOA data to find
the best fit. You could do this in two steps: first coarse, then fine.

- Try a different channel estimation function. 
- Apply filters to detect outliers and average the result of multiple pulses.

If you have completed this successfully, you can start integrating and estimating the car’s location. Start
by finding a set of suitable parameters for your audio beacon. Communicate with the other part of your
team. They should be finalizing the microphone recording code. If this is not the case, work together to
complete the code. It is better to complete the mid-term first with well-tested code and then to continue
working on more functionality. Otherwise, you will spend all your time trying to integrate all the loose
bits of code in the end.

## AFTER THE MIDTERM: REFERENCE SIGNAL AND INTEGRATION ##

As you will remember from your EE2T11 Telecommunications A practical in Q3 (?? @B update), you must have a clean
reference signal to get a good channel/impulse response estimate, which you can then use to deconvolve
the recordings you make to locate KITT. Getting this clean reference is what you will do here since it
entails more than just simulating the OOK code, as you must also consider the microphone’s channel
and the beacon’s behavior.
This reference is crucial in finding the channel’s impulse response between the beacon and the micro-
phone and, consequently, in finding the TDOA to locate KITT within the field.
Appendix A reminds you of the beacon signal parameters that are used to generate the beacon signal;
they are similar to what you saw in the EE2T11 practical.

## Reference Signal Assessment

**Task 1** Determine a good bit code to transmit using KITT’s beacon, for this consider its autocorrelation
function. You want a strong peak for the 0-lag of the autocorrelation but as low as possible for any
other lag.
*Hint:* Suitable codes could be randomly generated, or you can try some optimal codes (check
communication theory literature for “gold codes”).

**Task 2** Determine the other parameters for KITT. What is a good carrier frequency to use? What is a good bit frequency? A perfect repetition count is not yet required as this will depend on your later script but do make sure that the full code can be transmitted and that it can be recorded on all microphones within the same recording window.
*Hint:* You could try to find a datasheet of the microphones to determine a good carrier frequency;
it depends on the sensitivity of the loudspeaker and the microphones. More reliably: you can measure the response of the system for various carrier frequencies (e.g., spaced by 1000 Hz) and make a plot of the amplitude response. You can then later select a carrier frequency with maximal
response

**Task 3** Record the bit-code transmitted over KITT’s beacon with your new parameters. Keep the micro- phone very close to KITT’s beacon to get a recording that is as clear as possible, but do make sure
to avoid clipping. Look at the waveform you get from this recording. Can you think of a way to reduce the noise? If you cannot develop a way to do this, ask a TA or other instructors.

**Task 4** The recording you made with the microphone close to the beacon can be the reference signal.
Look at the waveform of the reference and look at its autocorrelation. What do you see? Compare this to the “ideal” OOK and autocorrelation from **Task 1**.

**Task 5** Clean up your recording and strip “zero intervals” away such that only a clean recording of a single pulse remains.

**Deliverable** Show the autocorrelations and plots you made in the final report and comment on what you see. Explain your answers to the questions, what led you to those answers, and explain your choices.

## Integration KITT assignments

**Task 1** Record several transmissions from KITT’s beacon with KITT at various locations in the field.
Store these recordings and KITT’s (x, y) coordinates.

**Task 2** Using the reference signal you developed previously and the algorithms you developed for the mid-term, estimate the locations using these recordings, off-line.

**Task 3** Do the same as in Task 2, but now couple the location algorithm and the recording code. Assuming you solve the “blocking recording” issue, you should now be able to drive KITT around and locate it all in realtime.

**Task 4** Try to add time-stamp information. By the time you calculated your position, KITT has already moved. And note that in your recording, you look for a pulse, which is also from some time ago.
Luckily, you can estimate most of these delays. Augmenting your location estimates with time stamps might be very helpful for the controller that your team will build next, in particular if you
intend to drive fast.

**Deliverable** Document this process and reflect on your findings and accuracies.